In [1]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [2]:
from google.colab import drive
drive.mount('/content/mnt')

Mounted at /content/mnt


In [48]:
result_dir = '/content/mnt/MyDrive/results/CoffeeIndex'

In [4]:
from glob import glob
for file in glob(f'{data_folder}/*.csv'):
    print(file)

/content/mnt/MyDrive/results/CoffeeIndex/스타벅스.csv
/content/mnt/MyDrive/results/CoffeeIndex/커피빈.csv
/content/mnt/MyDrive/results/CoffeeIndex/이디야.csv
/content/mnt/MyDrive/results/CoffeeIndex/빽다방.csv


In [49]:
starbucks = pd.read_csv(f'{result_dir}/스타벅스.csv')
coffeebean = pd.read_csv(f'{result_dir}/커피빈.csv')
ediya = pd.read_csv(f'{result_dir}/이디야.csv')
paik = pd.read_csv(f'{result_dir}/빽다방.csv')

In [45]:
starbucks.head()

,상호명,지점명,시도명,시군구명,도로명주소,ID
0,스타벅스,경남과기대점,경상남도,진주시,경상남도 진주시 동진로 34,진주
1,스타벅스,거제대명리조트점,경상남도,거제시,경상남도 거제시 일운면 거제대로 2660,거제
2,스타벅스,김해삼계점,경상남도,김해시,경상남도 김해시 가야로 182,김해
3,스타벅스,김해내동점,경상남도,김해시,경상남도 김해시 경원로 67,김해
4,스타벅스,경남대점,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 월영남로 3,창원 합포


In [8]:
len(starbucks.시군구명.unique()), len(coffeebean.시군구명.unique()), len(ediya.시군구명.unique()), len(paik.시군구명.unique())

(174, 87, 219, 152)

In [19]:
# 고성군 - 고성(강원), 고성(경남)
# 광역시 - 서울 용산, 서울 서대문, 대전 서구, 대전 유성, 세종
# 행정구 - 수원 장안, 용인 수지, 고양 일산동, 창원 합포, 창원 회원
tmp_gu_dict = {
    '수원': ['장안구', '권선구', '팔달구', '영통구'], 
    '성남': ['수정구', '중원구', '분당구'], 
    '안양': ['만안구', '동안구'], 
    '안산': ['상록구', '단원구'], 
    '고양': ['덕양구', '일산동구', '일산서구'], 
    '용인': ['처인구', '기흥구', '수지구'], 
    '청주': ['상당구', '서원구', '흥덕구', '청원구'], 
    '천안': ['동남구', '서북구'], 
    '전주': ['완산구', '덕진구'], 
    '포항': ['남구', '북구'], 
    '창원': ['의창구', '성산구', '진해구', '마산합포구', '마산회원구']
}

In [20]:
starbucks[starbucks.시도명.str.contains('세종|제주')].head(12).tail()

,상호명,지점명,시도명,시군구명,도로명주소,ID
588,스타벅스,세종첫마을점,세종특별자치시,세종특별자치시,세종특별자치시 세종특별자치시 한누리대로 169,세종
589,스타벅스,NaN,세종특별자치시,세종특별자치시,세종특별자치시 세종특별자치시 도움5로 19,세종
590,스타벅스,세종충남대학교병원점,세종특별자치시,세종특별자치시,세종특별자치시 세종특별자치시 보듬7로 20,세종
1479,스타벅스,제주송악산점,제주특별자치도,서귀포시,제주특별자치도 서귀포시 대정읍 형제해안로 322,서귀포
1480,스타벅스,성산일출봉점,제주특별자치도,서귀포시,제주특별자치도 서귀포시 성산읍 일출로 284-5,서귀포


In [46]:
metro_list = ['서울특별시','부산광역시','대구광역시','인천광역시','대전광역시','광주광역시','울산광역시']

def get_ID(df):
    si_name = [None] * len(df)

    for i in df.index:
        if df.시도명[i] in metro_list:
            if len(df.시군구명[i]) == 2:
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i]          # 시도명 + 시군구명 조합
            else:
                si_name[i] = df.시도명[i][:2] + ' ' + df.시군구명[i][:-1]     # 긴 구 이름에서 '구' 제외
        else:
            si_len = len(df.시군구명[i].split())
            if si_len == 1:
                if df.시군구명[i][:-1] == '고성':                             # 고성은 강원도/경남 구분
                    if df.시도명[i] == '강원도':
                        si_name[i] = '고성(강원)'
                    else:
                        si_name[i] = '고성(경남)'
                elif df.시군구명[i][:-1] in tmp_gu_dict.keys():               # 시군구명에 시 있을 때
                    if len(df.도로명주소[i].split()[2]) == 5:                 # 경상남도 창원시 마산합포구
                        si_name[i] = df.시군구명[i][:2] + ' ' + df.도로명주소[i].split()[2][2:-1]   # 창원 + 합포
                    else:
                        si_name[i] = df.시군구명[i][:2] + ' ' + df.도로명주소[i].split()[2][:-1]    # '구' 제외
                elif df.시군구명[i][:2] == '세종':                            # 세종특별자치시를 '세종'으로
                    si_name[i] = '세종'
                else:
                    si_name[i] = df.시군구명[i][:-1]

            else:
                _, admingu = df.시군구명[i].split()
                for key, values in tmp_gu_dict.items():
                    if admingu in values:
                        if len(admingu) == 2:
                            si_name[i] = key + ' ' + admingu
                        elif admingu in ['마산합포구', '마산회원구']:
                            si_name[i] = key + ' ' + admingu[2:-1]          # '마산합포구'를 '합포'로
                        else:
                            si_name[i] = key + ' ' + admingu[:-1]           # '구' 제외

    return si_name

In [22]:
starbucks['ID'] = get_ID(starbucks)
starbucks.head()

,상호명,지점명,시도명,시군구명,도로명주소,ID
0,스타벅스,경남과기대점,경상남도,진주시,경상남도 진주시 동진로 34,진주
1,스타벅스,거제대명리조트점,경상남도,거제시,경상남도 거제시 일운면 거제대로 2660,거제
2,스타벅스,김해삼계점,경상남도,김해시,경상남도 김해시 가야로 182,김해
3,스타벅스,김해내동점,경상남도,김해시,경상남도 김해시 경원로 67,김해
4,스타벅스,경남대점,경상남도,창원시 마산합포구,경상남도 창원시 마산합포구 월영남로 3,창원 합포


In [23]:
print(np.sort(starbucks.ID.unique()))
sb_set = set(starbucks.ID.unique())
print(len(sb_set))

['가평' '강릉' '거제' '경산' '경주' '계룡' '고령' '고성(강원)' '고양 덕양' '고양 일산동' '고양 일산서'
 '고창' '공주' '과천' '광명' '광양' '광주' '광주 광산' '광주 남구' '광주 동구' '광주 북구' '광주 서구'
 '구리' '구미' '군산' '군포' '김천' '김포' '김해' '나주' '남양주' '논산' '당진' '대구 남구' '대구 달서'
 '대구 달성' '대구 동구' '대구 북구' '대구 서구' '대구 수성' '대구 중구' '대전 대덕' '대전 동구' '대전 서구'
 '대전 유성' '대전 중구' '동두천' '동해' '목포' '무안' '문경' '밀양' '보령' '보은' '부산 강서' '부산 금정'
 '부산 기장' '부산 남구' '부산 동구' '부산 동래' '부산 부산진' '부산 북구' '부산 사상' '부산 사하' '부산 서구'
 '부산 수영' '부산 연제' '부산 영도' '부산 중구' '부산 해운대' '부안' '부천' '사천' '산청' '서귀포' '서산'
 '서울 강남' '서울 강동' '서울 강북' '서울 강서' '서울 관악' '서울 광진' '서울 구로' '서울 금천' '서울 노원'
 '서울 도봉' '서울 동대문' '서울 동작' '서울 마포' '서울 서대문' '서울 서초' '서울 성동' '서울 성북' '서울 송파'
 '서울 양천' '서울 영등포' '서울 용산' '서울 은평' '서울 종로' '서울 중구' '서울 중랑' '서천' '성남 분당'
 '성남 수정' '성남 중원' '세종' '속초' '수원 권선' '수원 영통' '수원 장안' '수원 팔달' '순천' '시흥' '아산'
 '안동' '안산 단원' '안산 상록' '안성' '안양 동안' '안양 만안' '양산' '양주' '양평' '여수' '여주' '영덕'
 '영양' '예산' '오산' '용인 기흥' '용인 수지' '용인 처인' '울산 남구' '울산 동구' '울산 북구' '울산 울주'
 '울산 중구' '원주' '음성' '의왕' '의정부' '이천' '익산' '인제' '인천 강화' 

In [24]:
ediya['ID'] = get_ID(ediya)
print(np.sort(ediya.ID.unique()))
ed_set = set(ediya.ID.unique())
print(len(ed_set))

['가평' '강릉' '강진' '거제' '거창' '경산' '경주' '고령' '고성(경남)' '고양 덕양' '고양 일산동'
 '고양 일산서' '고창' '고흥' '곡성' '공주' '과천' '광명' '광양' '광주' '광주 광산' '광주 남구' '광주 동구'
 '광주 북구' '광주 서구' '괴산' '구례' '구리' '구미' '군산' '군포' '금산' '김제' '김천' '김포' '김해'
 '나주' '남양주' '남원' '남해' '논산' '단양' '담양' '당진' '대구 남구' '대구 달서' '대구 달성' '대구 동구'
 '대구 북구' '대구 서구' '대구 수성' '대구 중구' '대전 대덕' '대전 동구' '대전 서구' '대전 유성' '대전 중구'
 '동두천' '동해' '목포' '무안' '무주' '문경' '밀양' '보령' '보은' '부산 강서' '부산 금정' '부산 기장'
 '부산 남구' '부산 동구' '부산 동래' '부산 부산진' '부산 북구' '부산 사상' '부산 사하' '부산 서구' '부산 수영'
 '부산 연제' '부산 영도' '부산 중구' '부산 해운대' '부안' '부여' '부천' '사천' '산청' '삼척' '상주' '서귀포'
 '서산' '서울 강남' '서울 강동' '서울 강북' '서울 강서' '서울 관악' '서울 광진' '서울 구로' '서울 금천'
 '서울 노원' '서울 도봉' '서울 동대문' '서울 동작' '서울 마포' '서울 서대문' '서울 서초' '서울 성동' '서울 성북'
 '서울 송파' '서울 양천' '서울 영등포' '서울 용산' '서울 은평' '서울 종로' '서울 중구' '서울 중랑' '서천'
 '성남 분당' '성남 수정' '성남 중원' '성주' '세종' '속초' '수원 권선' '수원 영통' '수원 장안' '수원 팔달'
 '순창' '순천' '시흥' '아산' '안동' '안산 단원' '안산 상록' '안성' '안양 동안' '안양 만안' '양구' '양산'
 '양양' '양주' '양평' '여수' '여주' '연천' '영광' '영덕' '영동' '영암' '영월' '영주

In [25]:
coffeebean['ID'] = get_ID(coffeebean)
print(np.sort(coffeebean.ID.unique()))
cb_set = set(coffeebean.ID.unique())
print(len(cb_set))

['경주' '고양 덕양' '고양 일산동' '고양 일산서' '광명' '광주 남구' '광주 동구' '광주 서구' '구미' '군포'
 '금산' '김포' '김해' '남양주' '대구 수성' '대구 중구' '대전 서구' '대전 유성' '동해' '부산 강서' '부산 기장'
 '부산 동구' '부산 부산진' '부산 북구' '부산 수영' '부산 중구' '부산 해운대' '부천' '상주' '서울 강남'
 '서울 강동' '서울 강북' '서울 강서' '서울 관악' '서울 광진' '서울 구로' '서울 금천' '서울 노원' '서울 동대문'
 '서울 동작' '서울 마포' '서울 서대문' '서울 서초' '서울 성동' '서울 성북' '서울 송파' '서울 양천' '서울 영등포'
 '서울 용산' '서울 은평' '서울 종로' '서울 중구' '서울 중랑' '성남 분당' '성남 수정' '성남 중원' '수원 권선'
 '수원 영통' '수원 장안' '수원 팔달' '안양 동안' '안양 만안' '여주' '연천' '영덕' '용인 기흥' '용인 수지'
 '울산 남구' '울산 중구' '울진' '원주' '이천' '인천 남동' '인천 부평' '인천 연수' '인천 중구' '전주 완산'
 '제주' '제천' '진주' '창원 성산' '창원 의창' '창원 진해' '창원 합포' '천안 동남' '청주 상당' '청주 흥덕'
 '통영' '파주' '평택' '포항 북구' '하남' '함안' '해남' '화성']
95


In [26]:
paik['ID'] = get_ID(paik)
print(np.sort(paik.ID.unique()))
bd_set = set(paik.ID.unique())
print(len(bd_set))

['강릉' '거창' '경산' '경주' '고성(경남)' '고양 덕양' '고양 일산동' '고양 일산서' '고창' '공주' '광명'
 '광양' '광주' '광주 광산' '광주 남구' '광주 동구' '광주 북구' '광주 서구' '구리' '구미' '군산' '군포'
 '김제' '김포' '김해' '남양주' '남원' '논산' '당진' '대구 달서' '대구 달성' '대구 동구' '대구 북구'
 '대구 서구' '대구 수성' '대구 중구' '대전 대덕' '대전 동구' '대전 서구' '대전 유성' '대전 중구' '동두천'
 '동해' '목포' '무안' '문경' '보령' '부산 금정' '부산 남구' '부산 동래' '부산 부산진' '부산 북구' '부산 사하'
 '부산 중구' '부산 해운대' '부안' '부천' '삼척' '서귀포' '서산' '서울 강남' '서울 강동' '서울 강북'
 '서울 강서' '서울 관악' '서울 광진' '서울 구로' '서울 금천' '서울 노원' '서울 도봉' '서울 동대문' '서울 동작'
 '서울 마포' '서울 서대문' '서울 서초' '서울 성동' '서울 성북' '서울 송파' '서울 영등포' '서울 용산' '서울 은평'
 '서울 종로' '서울 중구' '서울 중랑' '성남 분당' '성남 수정' '성남 중원' '세종' '수원 권선' '수원 영통'
 '수원 장안' '수원 팔달' '순천' '시흥' '아산' '안동' '안산 단원' '안산 상록' '안양 동안' '안양 만안' '양산'
 '양주' '양평' '여수' '영동' '영암' '영주' '예산' '예천' '오산' '옥천' '용인 기흥' '용인 수지' '용인 처인'
 '울산 남구' '울산 동구' '울산 북구' '울진' '원주' '의왕' '의정부' '이천' '익산' '인천 강화' '인천 계양'
 '인천 남동' '인천 동구' '인천 미추홀' '인천 부평' '인천 서구' '인천 연수' '인천 중구' '전주 덕진' '전주 완산'
 '정읍' '제주' '증평' '진안' '진주' '진천' '창원 성산' '창원 의창' '창원 합포' '천안 동남

In [28]:
# 합집합
city_set = sb_set | ed_set | cb_set | bd_set
len(city_set)

241

In [30]:
# 차집합
sb_set - ed_set

{'계룡', '고성(강원)', '영양', '청양', '하동'}

In [31]:
starbucks[starbucks.ID.isin({'계룡', '고성(강원)', '영양', '청양', '하동'})]

,상호명,지점명,시도명,시군구명,도로명주소,ID
35,스타벅스,NaN,경상남도,하동군,경상남도 하동군 하동읍 중앙로 12,하동
530,스타벅스,NaN,경상북도,영양군,경상북도 영양군 석보면 원리길 11-3,영양
1411,스타벅스,대명델피노리조트점,강원도,고성군,강원도 고성군 토성면 미시령옛길 1153,고성(강원)
1448,스타벅스,NaN,강원도,고성군,강원도 고성군 간성읍 간성로 36,고성(강원)
1541,스타벅스,NaN,충청남도,청양군,충청남도 청양군 청양읍 중앙로 116-1,청양
1542,스타벅스계룡대로DT점,계룡대로DT점,충청남도,계룡시,충청남도 계룡시 엄사면 번영로 6,계룡


In [32]:
paik[paik.ID.isin({'계룡', '고성(강원)', '영양', '청양', '하동'})]

,상호명,지점명,시도명,시군구명,도로명주소,ID
22,빽다방,하동점,경상남도,하동군,경상남도 하동군 하동읍 경서대로 136,하동


In [33]:
pt_sb = starbucks.pivot_table('상호명', index='ID', aggfunc='count')
pt_sb.head()

,상호명
ID,
가평,2
강릉,5
거제,6
경산,8
경주,8


In [34]:
starbucks[['상호명','ID']].groupby('ID').count().head()

,상호명
ID,
가평,2
강릉,5
거제,6
경산,8
경주,8


In [35]:
pt_sb.columns = ['스타벅스']
pt_sb.reset_index(inplace=True)
pt_sb.head()

,ID,스타벅스
0,가평,2
1,강릉,5
2,거제,6
3,경산,8
4,경주,8


In [36]:
pt_cb = coffeebean.pivot_table('상호명', index='ID', aggfunc='count')
pt_cb.columns = ['커피빈']
pt_cb.reset_index(inplace=True)
pt_cb.head()

,ID,커피빈
0,경주,2
1,고양 덕양,1
2,고양 일산동,5
3,고양 일산서,5
4,광명,1


In [37]:
pt_ed = ediya.pivot_table('상호명', index='ID', aggfunc='count')
pt_ed.columns = ['이디야']
pt_ed.reset_index(inplace=True)
pt_ed.head()

,ID,이디야
0,가평,9
1,강릉,15
2,강진,1
3,거제,9
4,거창,1


In [38]:
pt_bd = paik.pivot_table('상호명', index='ID', aggfunc='count')
pt_bd.columns = ['빽다방']
pt_bd.reset_index(inplace=True)
pt_bd.head()

,ID,빽다방
0,강릉,2
1,거창,1
2,경산,4
3,경주,1
4,고성(경남),2


- 커피지수 구하기

In [40]:
cf = pd.merge(pt_ed, pt_sb, how='left')
len(cf)

236

In [41]:
cf = pd.merge(cf, pt_cb, how='left')
cf = pd.merge(cf, pt_bd, how='left')
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2.0,NaN,NaN
1,강릉,15,5.0,NaN,2.0
2,강진,1,NaN,NaN,NaN
3,거제,9,6.0,NaN,NaN
4,거창,1,NaN,NaN,1.0


In [42]:
cf.fillna(0, inplace=True)
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2.0,0.0,0.0
1,강릉,15,5.0,0.0,2.0
2,강진,1,0.0,0.0,0.0
3,거제,9,6.0,0.0,0.0
4,거창,1,0.0,0.0,1.0


In [43]:
cf['스타벅스'] = cf.스타벅스.astype(int)
cf['커피빈'] = cf.커피빈.astype(int)
cf['빽다방'] = cf.빽다방.astype(int)
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방
0,가평,9,2,0,0
1,강릉,15,5,0,2
2,강진,1,0,0,0
3,거제,9,6,0,0
4,거창,1,0,0,1


In [44]:
cf['커피지수'] = (cf.스타벅스 + cf.커피빈) / (cf.이디야 + cf.빽다방)
cf.head()

,ID,이디야,스타벅스,커피빈,빽다방,커피지수
0,가평,9,2,0,0,0.222222
1,강릉,15,5,0,2,0.294118
2,강진,1,0,0,0,0.000000
3,거제,9,6,0,0,0.666667
4,거창,1,0,0,1,0.000000


In [50]:
cf.to_csv(f'{result_dir}/커피지수.csv', index=False)